In [ ]:

# IMPORTANT: RUN THIS CELL IN ORDER TO IMPORT YOUR KAGGLE DATA SOURCES
# TO THE CORRECT LOCATION (/kaggle/input) IN YOUR NOTEBOOK,
# THEN FEEL FREE TO DELETE THIS CELL.
# NOTE: THIS NOTEBOOK ENVIRONMENT DIFFERS FROM KAGGLE'S PYTHON
# ENVIRONMENT SO THERE MAY BE MISSING LIBRARIES USED BY YOUR
# NOTEBOOK.

import os
import sys
from tempfile import NamedTemporaryFile
from urllib.request import urlopen
from urllib.parse import unquote, urlparse
from urllib.error import HTTPError
from zipfile import ZipFile
import tarfile
import shutil

CHUNK_SIZE = 40960
DATA_SOURCE_MAPPING = 'miniplaces:https%3A%2F%2Fstorage.googleapis.com%2Fkaggle-data-sets%2F1271856%2F2119650%2Fbundle%2Farchive.zip%3FX-Goog-Algorithm%3DGOOG4-RSA-SHA256%26X-Goog-Credential%3Dgcp-kaggle-com%2540kaggle-161607.iam.gserviceaccount.com%252F20240802%252Fauto%252Fstorage%252Fgoog4_request%26X-Goog-Date%3D20240802T214313Z%26X-Goog-Expires%3D259200%26X-Goog-SignedHeaders%3Dhost%26X-Goog-Signature%3D4bfbace518c7a9c5d18389afff5737e05b8a15107d37afb9640af2dea70024f7dda8e53ff8099dece1b0532677a6e2a8eeda5c7f2d6090500a5d3e2a74b7dc846b03b15aadb137b8a832794b197dcceefc3bcadb1765c05321d0f00e9c3e7d1f231bd29fa731197e28007146c5b92fcdf1d572bd1b37cf2c99c9d8f174cac3268d97cac492d5b699f3797f0e1b4b8e310dfcef00e0930bc07b21d28798675c1576a447527e963bb8402e890e281612d8402782db9922cb9b9d66b02d06af62b9288c1c0cda0e393540598f1241124be013015db0aa1f3d52da5c22ffc6e6e7016cc0290ac462b2c9c7b23d50fd1345b7e64ff8e1be908b20b9223191bb8447d9'

KAGGLE_INPUT_PATH='/kaggle/input'
KAGGLE_WORKING_PATH='/kaggle/working'
KAGGLE_SYMLINK='kaggle'

!umount /kaggle/input/ 2> /dev/null
shutil.rmtree('/kaggle/input', ignore_errors=True)
os.makedirs(KAGGLE_INPUT_PATH, 0o777, exist_ok=True)
os.makedirs(KAGGLE_WORKING_PATH, 0o777, exist_ok=True)

try:
  os.symlink(KAGGLE_INPUT_PATH, os.path.join("..", 'input'), target_is_directory=True)
except FileExistsError:
  pass
try:
  os.symlink(KAGGLE_WORKING_PATH, os.path.join("..", 'working'), target_is_directory=True)
except FileExistsError:
  pass

for data_source_mapping in DATA_SOURCE_MAPPING.split(','):
    directory, download_url_encoded = data_source_mapping.split(':')
    download_url = unquote(download_url_encoded)
    filename = urlparse(download_url).path
    destination_path = os.path.join(KAGGLE_INPUT_PATH, directory)
    try:
        with urlopen(download_url) as fileres, NamedTemporaryFile() as tfile:
            total_length = fileres.headers['content-length']
            print(f'Downloading {directory}, {total_length} bytes compressed')
            dl = 0
            data = fileres.read(CHUNK_SIZE)
            while len(data) > 0:
                dl += len(data)
                tfile.write(data)
                done = int(50 * dl / int(total_length))
                sys.stdout.write(f"\r[{'=' * done}{' ' * (50-done)}] {dl} bytes downloaded")
                sys.stdout.flush()
                data = fileres.read(CHUNK_SIZE)
            if filename.endswith('.zip'):
              with ZipFile(tfile) as zfile:
                zfile.extractall(destination_path)
            else:
              with tarfile.open(tfile.name) as tarfile:
                tarfile.extractall(destination_path)
            print(f'\nDownloaded and uncompressed: {directory}')
    except HTTPError as e:
        print(f'Failed to load (likely expired) {download_url} to path {destination_path}')
        continue
    except OSError as e:
        print(f'Failed to load {download_url} to path {destination_path}')
        continue

print('Data source import complete.')


In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
from collections import OrderedDict
from torch.nn import Linear, ReLU, Sequential

mlp = torch.nn.Sequential(OrderedDict([
    ('layer1', Sequential(Linear(2, 20), ReLU())),
    ('layer2', Sequential(Linear(20, 20), ReLU())),
    ('layer3', Sequential(Linear(20, 2)))
]))

print(mlp)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
from matplotlib import pyplot as plt

def visualize_net(net, classify_target):
    fig, (ax1, ax2) = plt.subplots(1, 2)
    grid = torch.stack([
        torch.linspace(-2, 2, 100)[None, :].expand(100, 100),
        torch.linspace(2, -2, 100)[:, None].expand(100, 100),
    ])
    x, y = grid
    target = classify_target(x, y)
    ax1.set_title('target')
    ax1.imshow(target.float(), cmap='hot', extent=[-2,2,-2,2])
    ax2.set_title('network output')
    score = net(grid.permute(1, 2, 0).reshape(-1, 2).cuda()).softmax(1)
    ax2.imshow(score[:,1].reshape(100, 100).detach().cpu(), cmap='hot', extent=[-2,2,-2,2])
    plt.show()


In [ ]:
from torch.nn.functional import cross_entropy
from torch.optim import Adam

def classify_target(x, y):
  return (y.floor() == x.floor()).long()

mlp.cuda()
optimizer = Adam(mlp.parameters(), lr=.01)
for iteration in range(1024):
  in_batch = torch.randn(10000, 2, device='cuda')
  target_batch = classify_target(in_batch[:,0], in_batch[:,1])
  out_batch = mlp(in_batch)
  loss = cross_entropy(out_batch, target_batch)
  if iteration > 0:
    mlp.zero_grad()
    loss.backward()
    optimizer.step()
  if iteration == 2 ** iteration.bit_length() - 1:
    pred_batch = out_batch.max(1)[1]
    accuracy = (pred_batch == target_batch).float().sum().item() / len(in_batch)
    print(f'iteration {iteration} accuracy {accuracy}')
    visualize_net(mlp, classify_target)

### A network can be saved by saving its `state_dict` since it gathers together all the params of the submodule it can save it all at once.

### If we wanted to load just a subset of layers, we could do that by picking out keys of the dictionary and adjusting their names by hand.

In [ ]:
for k, v, in mlp.state_dict().items():
    print(f'{k}: {v.dtype}{tuple(v.shape)}')

torch.save(mlp.state_dict(), 'checkpoints/mlp.pth')


### Defining `forward` to create custom NeuralNets 🤓

#### Sometimes you will want to hookup network components in a more complicated way than just sequential operationof predefined components

#### For eg in ResNets we learn perturbations (where we compare a known solution w/ an unknown
#### one and try to solve this way) of the identity. Have a layer learn to compute small residual instead of the whole total answer.

![image.png](attachment:416bc8f0-109b-4c71-8838-31fd1c9a05b6.png)![image.png](attachment:6b8b0a7b-5c50-407a-911a-3bbd0d657bd0.png)

##### To apply the residual trick to our 3 layer network, wel define the operation by writing
##### our own `forward` function to look like this -

In [ ]:
class myneuralnet(torch.nn.Module):
    def __init__(self):
        super().__init__()
        self.layer1 = Sequential(Linear(2, 20), ReLU())
        self.residual_layer2 = Sequential(Linear(20, 20), ReLU())
        self.layer3 = Linear(20, 2)

    def forward(self, x):
        x = self.layer1(x)
        x = x + self.residual_layer2(x)
        x = self.layer3(x)
        return x

res_mlp = myneuralnet()
print(res_mlp)



In [ ]:
from torch.nn.functional import cross_entropy
from torch.optim import Adam

def classify_target(x, y):
  return (y.floor() == x.floor()).long() ### grid img

res_mlp.cuda()
optimizer = Adam(res_mlp.parameters(), lr=.01)
for iteration in range(1024):
  in_batch = torch.randn(10000, 2, device='cuda')
  target_batch = classify_target(in_batch[:,0], in_batch[:,1])
  out_batch = res_mlp(in_batch)
  loss = cross_entropy(out_batch, target_batch)
  if iteration > 0:
    res_mlp.zero_grad()
    loss.backward()
    optimizer.step()
  if iteration == 2 ** iteration.bit_length() - 1:
    pred_batch = out_batch.max(1)[1]
    accuracy = (pred_batch == target_batch).float().sum().item() / len(in_batch)
    print(f'iteration {iteration} accuracy {accuracy}')
    visualize_net(res_mlp, classify_target)

### Different loss functions in Pytorch 😊

In [ ]:
import torch
import torch.nn as nn

input = torch.randn(3, 5, requires_grad=True)
target = torch.randn(3, 5)

mae_loss = nn.L1Loss()
output = mae_loss(input, target)
output.backward()

print('input: ', input)
print('target: ', target)
print('output: ', output)

In [ ]:
torch.nn.MSELoss

In [ ]:
torch.nn.NLLLoss

In [ ]:
torch.nn.CrossEntropyLoss

In [ ]:
torch.nn.HingeEmbeddingLoss


In [ ]:
torch.nn.MarginRankingLoss

In [ ]:
torch.nn.TripletMarginLoss

In [ ]:
torch.nn.KLDivLoss

#### More on the different loss functions here: https://neptune.ai/blog/pytorch-loss-functions

In [ ]:
ls datasets/miniplaces/val/golf_course

### Constructing an Imagefolder in Pytorch 🌇🌆

In [ ]:
import torch
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import os

### Define the path to the dataset (kaggle dataset path)

dataset_path = '/kaggle/input/miniplaces/data/val'

### Define transformation operations for more clarity

transform = transforms.Compose([
    transforms.ToTensor()
])

### Load the dataset using ImageFolder
val_set = datasets.ImageFolder(root=dataset_path, transform=transform)
print('Length is', len(val_set))


## Get the 51000th item from the dataset
item = val_set[5100]
print('5100th item is a pair', item)

### Display the class name of the 5100th item
print('Class name is', val_set.classes[item[1]])

## Display the image

to_pil = ToPILImage()
image = to_pil(item[0])

# Display the image
image = transforms.ToPILImage()(item[0])
plt.imshow(image)
plt.title(val_set.classes[item[1]])
plt.show()

### Transforming a PIL image into a pytorch tensor 🌇🔥

### A pil image is not convenient for training - we would prefer our data to return to torch tensor.
### So we tell `ImageFolder` to do this by specifying transform function on construction.
### Pytorch comes with a standard transform function also `torchvision.transforms.ToTensor()`

In [ ]:
import time

print('Going over the dataset as an array')
start = time.time()
summed_image_dataset = 0
batch_size = 100
for i in range(0, len(val_set), batch_size):
    image_batch = torch.stack([val_set[i+j][0] for j in range(batch_size)])
    summed_image_dataset += image_batch.sum(0)
end = time.time()
print(f'Took {end - start} seconds')

### using torch.utils

print('Going over the same dataset using a dataloader')

start = time.time()
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, num_workers=10)
summed_image_loader = 0
for image_batch, label_batch in val_loader:
    summed_image_loader += image_batch.sum(0)
end = time.time()
print(f'Took {end - start} seconds')

print('Numerical difference is exactly', (summed_image_loader - summed_image_dataset).norm().item())


#### yea so its substantialy better to use `torch.utils()` 😅

In [ ]:
### Exercise

## Try and alter -

# 1. num_workers and note the changes in speed
# 2. batch_size and note the changes in speed.


import time

print('Going over the dataset as an array')
start = time.time()
summed_image_dataset = 0
batch_size = 100
for i in range(0, len(val_set), batch_size):
    image_batch = torch.stack([val_set[i+j][0] for j in range(batch_size)])
    summed_image_dataset += image_batch.sum(0)
end = time.time()
print(f'Took {end - start} seconds')

### using torch.utils

print('Going over the same dataset using a dataloader')

start = time.time()
val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=batch_size, num_workers=10)
summed_image_loader = 0
for image_batch, label_batch in val_loader:
    summed_image_loader += image_batch.sum(0)
end = time.time()
print(f'Took {end - start} seconds')

print('Numerical difference is exactly', (summed_image_loader - summed_image_dataset).norm().item())


### Other common dataloader tricks. `DataLoader` can do a few more useful things.

-  Although a `DataLoader` does not put batches on the GPU directly (because of multithreading limitations), it can put the batch in pinned memory, which is faster to copy to the GPU later after you get it out of the DataLoader. Make the DataLoader with `pin_memory=True` for this.

- During training you usually do not want the batches in alphabetical order. The DataLoader can shuffle the batches so that they are randomized, instead of sequential. `shuffle=True` for this.

### Now wel train a classifier, using adam optim and a simplified version of Res-Net 18 neuralnet 🔥

In [ ]:

import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import os
from tqdm import tqdm

from tqdm import tqdm

## Create a dataset of miniplaces training images

### Define the path to the dataset (kaggle dataset path)

dataset_path = '/kaggle/input/miniplaces/data/val'

### Define transformation operations for more clarity

### instead of `.Compose()` this could also simply be `.torchvision`
transform = transforms.Compose([
    transforms.ToTensor()
])

### Load the dataset using ImageFolder
train_set = datasets.ImageFolder(root=dataset_path, transform=transform)
print('Length is', len(train_set))

### Wrapping the dataset in a high-speed DataLoader w batch_size 100

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=100, num_workers=10,
    shuffle=True,
    pin_memory=True
)

### Creating an untrained neuralnet using the ResNet 18 architecture

model = torchvision.models.resnet18(num_classes=100).cuda()


### Set up the model for training using the Adam Optim
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Now training, optimizing an objective on batches
### Here we look at the image once

for batch in tqdm(train_loader):
    images, labels = [d.cuda() for d in batch]
    optimizer.zero_grad()
    scores = model(images.cuda())
    loss = torch.nn.functional.cross_entropy(scores, labels)
    loss.backward()
    optimizer.step()


### Checking accuracy w a heldout Dataset 🤓

### Now we can do this easily by greating a second `ImageFolder` dataset and `DataLoader` with a second set
### of images not used for training.

In [ ]:
import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import os
from tqdm import tqdm

from tqdm import tqdm

## Create a dataset of miniplaces training images

### Define the path to the dataset (kaggle dataset path)

dataset_path = '/kaggle/input/miniplaces/data/val'

### Define transformation operations for more clarity

### instead of `.Compose()` this could also simply be `.torchvision`
transform = transforms.Compose([
    transforms.ToTensor()
])

### Load the dataset using ImageFolder
val_set = datasets.ImageFolder(root=dataset_path, transform=transform)
print('Length is', len(train_set))

### Wrapping the dataset in a high-speed DataLoader w batch_size 100

val_loader = torch.utils.data.DataLoader(
    val_set, batch_size=100, num_workers=10,
    shuffle=True,
    pin_memory=True
)

### Now this function runs over validation images and counts accurate preds :>

def accuracy():
    model.eval()
    correct = 0
    for iter, batch in enumerate(val_loader):
        images, labels = [d.cuda() for d in batch]
        with torch.no_grad():
            scores = model(images.cuda())
        correct += (scores.max(1)[1] == labels).float().sum()
    return correct.item() / len(val_set)

print(f'Accuracy on unseen images {accuracy() * 100}% (random guesses would be 1%)')

### Improving our model with Data Augmentation 😅

### For example if we randomly adjust the crop, color, or orientation of the image while loading, using the same image file multiple times will produce different training examples for the network.

### This is an easy way to increase the amount of training diversity in the data set without requring more actual images.

### To do data augmentation in a pytorch Dataset, you can specify more operations on `transform=` besides `ToTensor()`

In [ ]:
### Using Data Augmentation to try to improve our model :>

import torch
from torchvision import datasets, transforms, models
import matplotlib.pyplot as plt
from torchvision.transforms import ToPILImage
import os
from tqdm import tqdm

from tqdm import tqdm

## Create a dataset of miniplaces training images

### Define the path to the dataset (kaggle dataset path)

dataset_path = '/kaggle/input/miniplaces/data/val'

### Define transformation operations for more clarity

### instead of `.Compose()` this could also simply be `.torchvision`
transform = transforms.Compose([
    transforms.RandomCrop(112),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor()
])

### Load the dataset using ImageFolder
train_set = datasets.ImageFolder(root=dataset_path, transform=transform)
print('Length is', len(train_set))

### Wrapping the dataset in a high-speed DataLoader w batch_size 100

train_loader = torch.utils.data.DataLoader(
    train_set, batch_size=100, num_workers=10,
    shuffle=True,
    pin_memory=True
)

### Creating an untrained neuralnet using the ResNet 18 architecture

#model = torchvision.models.resnet18(num_classes=100).cuda()

### Set up the model for training using the Adam Optim
model.train()
optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

### Now training, optimizing an objective on batches
### Here we look at the image once

for batch in tqdm(train_loader):
    images, labels = [d.cuda() for d in batch]
    optimizer.zero_grad()
    scores = model(images.cuda())
    loss = torch.nn.functional.cross_entropy(scores, labels)
    loss.backward()
    optimizer.step()
print(f'Accuracy on unseen images {accuracy() * 100}% (random guesses would be 1%)')
